In [1]:
#! pip uninstall -y langchain
#!pip install langchain langchain-openai

# Retrieval Augmented Generation avec Langchain

Seuls, les LLMs ont peuvent avoir un problème de rafraichissement des données.

Le monde des LLM est figé dans le temps. Leur monde existe comme un instantané statique du monde tel qu'il était dans leurs données de formation.

Une solution à ce problème est l'"augmentation de la récupération" (Retrieval Augmentation). L'idée sous-jacente est que nous récupérons des informations pertinentes à partir d'une base de connaissances externe et que nous donnons ces informations à notre LLM.

In [2]:
import os
OPENAI_API_KEY= "YOUR_API_KEY"
os.environ['OPENAI_API_KEY']= OPENAI_API_KEY

In [3]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

### Construire une BD de connaissance vectorisé

Facebook AI Similarity Search (Faiss) est une bibliothèque pour la recherche efficace de similarités et le regroupement de vecteurs denses. Elle contient des algorithmes qui recherchent dans des ensembles de vecteurs de toute taille, jusqu'à ceux qui ne tiennent pas dans la mémoire vive. Elle contient également un code de soutien pour l'évaluation et le réglage des paramètres.

Nous pouvons également convertir le vectorstore en une classe Retriever. Cela nous permet de l'utiliser facilement dans d'autres méthodes LangChain, qui fonctionnent en grande partie avec des retrievers.

In [4]:
vectorstore = FAISS.from_texts(
    ["Bryan travaille dans la cuisine"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

#prompt template
template = """Réponds à la question en te basant uniquement sur le contexte suivant:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [7]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
chain.invoke("Où est Bryan?")

'Bryan est dans la cuisine.'

In [8]:
template = """Réponds à la question en te basant uniquement sur le contexte suivant:
{context}

Question: {question}

Réponds dans la langue suivante: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [38]:
chain.invoke({"question": "Où travaille Bryan?", "language": "italien"})

'Bryan lavora in cucina.'

# Chaîne de Retrieval Consersationel

In [11]:
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate

_template = """À partir de la conversation suivante et d'une question de suivi, reformulez la question de suivi pour en faire une question indépendante, dans sa langue d'origine.

Historique du chat:
{chat_history}
Entrée de suivi: {question}
Question indépendante:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [12]:
template = """Réponds à la question en te basant uniquement sur le contexte suivant:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [13]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [14]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [15]:
conversational_qa_chain.invoke(
    {
        "question": "Où est Bryan?",
        "chat_history": [],
    }
)

AIMessage(content='Bryan se trouve actuellement dans la cuisine.')

In [16]:
conversational_qa_chain.invoke(
    {
        "question": "Et où a-t-il cuisiné?",
        "chat_history": [
            HumanMessage(content="Qui a fait ce gateau?"),
            AIMessage(content="Bryan."),
        ],
    }
)

AIMessage(content='Bryan a cuisiné le gâteau dans la cuisine.')

## Avec Mémoire et en retournant le document source

In [35]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [36]:
inputs = {"question": "Où est-ce que Bryan a travaillé?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Bryan a travaillé dans la cuisine.'),
 'docs': [Document(page_content='Bryan travaille dans la cuisine')]}

Notez que la mémoire n'est pas sauvegardée automatiquement

Pour l'instant, vous devez la sauvegarder vous-même

In [37]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [38]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Où est-ce que Bryan a travaillé?'),
  AIMessage(content='Bryan a travaillé dans la cuisine.')]}

On peut même rajouter du contexte externe ensuite

In [33]:
#On peut même rajouter du contexte dans L,historique de chat
chat_history= [
            HumanMessage(content="Qui a fait ce gateau?"),
            AIMessage(content="Bryan a fait un gateau au chocolat."),
        ]

In [41]:
memory.save_context({"question":chat_history[0].content}, {"answer": chat_history[1].content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Où est-ce que Bryan a travaillé?'),
  AIMessage(content='Bryan a travaillé dans la cuisine.'),
  HumanMessage(content='Qui a fait ce gateau?'),
  AIMessage(content='Bryan a fait un gateau au chocolat.')]}

In [43]:
inputs = {"question": "A-t-il fait un gateau?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Bryan a fait un gâteau au chocolat.'),
 'docs': [Document(page_content='Bryan travaille dans la cuisine')]}